# Get Data Ready

In [ ]:
#!pip install gdown

In [ ]:
#https://drive.google.com/file/d/1hTp8qDF4r2tTk7W6SBnfYyRoNwwSdkap/view?usp=sharing
!gdown --id 1hTp8qDF4r2tTk7W6SBnfYyRoNwwSdkap
#https://drive.google.com/file/d/1T-yZGWyPC89SCCbm1m0DrRMGyDlUaonl/view?usp=sharing
!gdown --id 1T-yZGWyPC89SCCbm1m0DrRMGyDlUaonl

In [26]:
import pickle

token_file = open("danmu_token_main.pkl", "rb")
dist_file = open("danmu_dist_main.pkl", "rb")

tokens = pickle.load(token_file)
dists = pickle.load(dist_file)

print(len(tokens))
print(len(dists))

337392
337392


In [27]:
import numpy as np
labels = [np.argmax(i) for i in dists]

label_distribution = {}
for i in labels:
    if i in label_distribution:
        label_distribution[i] += 1
    else:
        label_distribution[i] = 1

In [28]:
len(label_distribution)

19

In [29]:
# remove label with no data
label_list = list(label_distribution.keys())
labels = [label_list.index(i) for i in labels]

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.DataFrame({
    'text': tokens,
    'label': labels
})

train_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.1)

In [31]:
#Debugging
train_df = train_df[train_df['text'] != 'nan']
test_df = test_df[test_df['text'] != 'nan']
val_df = val_df[val_df['text'] != 'nan']


In [32]:
train_df.reset_index(drop=True)
test_df.reset_index(drop=True)
val_df.reset_index(drop=True)

,text,label
0,那是脐带哥哥,10
1,原来有人脉,8
2,想去了,1
3,百香果射手,1
4,中国政法大学,6
...,...,...
26987,匹克嗯,12
26988,321看钉宫,8
26989,露露哇露露打,4
26990,这是臭虫吧,0


In [33]:
#save to csv

train_df = train_df.dropna()
test_df = test_df.dropna()
val_df = val_df.dropna()

train_df.to_csv('data_csv/train.csv', index = False)
test_df.to_csv('data_csv/test.csv', index = False)
val_df.to_csv('data_csv/val.csv', index = False)

# train_dict = train_df.to_dict()
# test_dict = test_df.to_dict()
# val_dict = val_df.to_dict()
# train_data_json = {
#     'train' : train_dict,
#     'test' : test_dict,
#     'validation': val_dict
# }

# import json

# with open('data_csv/data.json', 'w') as f:
#     json.dump(train_data_json, f)
    

# Preprocessing

In [34]:
import datasets
train = datasets.Dataset.from_pandas(train_df, preserve_index=False)
val = datasets.Dataset.from_pandas(val_df, preserve_index=False)
test = datasets.Dataset.from_pandas(test_df, preserve_index=False)

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [37]:
#Tokenization

tokenized_train =  train.map(preprocess_function, batched=True)
tokenized_val =  val.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

  0%|          | 0/243 [00:00<?, ?ba/s]

  0%|          | 0/27 [00:00<?, ?ba/s]

  0%|          | 0/68 [00:00<?, ?ba/s]

In [38]:
# For batching and padding
import transformers 

data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
#!conda install pytorch torchvision torchaudio cpuonly -c pytorch

# Model

In [ ]:
# model = transformers.AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-jd-full-chinese",\
#                                                                         num_labels = len(label_distribution), \
#                                                                        ignore_mismatched_sizes=True)
model = transformers.AutoModelWithLMHead.from_pretrained("t5-small")
                                                                        #num_labels = len(label_distribution))

training_args = transformers.TrainingArguments(
    output_dir="t5_small",
    
)

loading configuration file https://huggingface.co/uer/chinese_roberta_L-12_H-768/resolve/main/config.json from cache at /home/ec2-user/.cache/huggingface/transformers/0581d850743cad42501488567cceb8dc9ce50d9f05ad632d273b4389b2f52f68.042085124aedc502028136283b7bf9a169a238009bd6c309f049b249216061a2
Model config BertConfig {
  "_name_or_path": "uer/chinese_roberta_L-12_H-768",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    

Downloading:   0%|          | 0.00/390M [00:00<?, ?B/s]

storing https://huggingface.co/uer/chinese_roberta_L-12_H-768/resolve/main/pytorch_model.bin in cache at /home/ec2-user/.cache/huggingface/transformers/8150d6a23b0f518134caba5b6141414848a8adbe069f59c18ddbdd9b7498045c.a61653c00cec4d0c4b7281005a15c05b7277d845fb82d1967138f8296cc7622b
creating metadata file for /home/ec2-user/.cache/huggingface/transformers/8150d6a23b0f518134caba5b6141414848a8adbe069f59c18ddbdd9b7498045c.a61653c00cec4d0c4b7281005a15c05b7277d845fb82d1967138f8296cc7622b
loading weights file https://huggingface.co/uer/chinese_roberta_L-12_H-768/resolve/main/pytorch_model.bin from cache at /home/ec2-user/.cache/huggingface/transformers/8150d6a23b0f518134caba5b6141414848a8adbe069f59c18ddbdd9b7498045c.a61653c00cec4d0c4b7281005a15c05b7277d845fb82d1967138f8296cc7622b


In [16]:
#metrics

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [17]:
trainer = transformers.Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_val,
    data_collator=data_collator,
    tokenizer = tokenizer,
)

In [18]:
#trainer.train()

# SageMaker

In [19]:
import sagemaker

session = sagemaker.Session()

session_bucket = None
if session_bucket == None and session is not None:
    session_bucket = session.default_bucket()
    
role = sagemaker.get_execution_role()
session = sagemaker.Session(default_bucket = session_bucket)

print(f'role: {role}')
print(f'bucket: {session_bucket}')
print(f'session region: {session.boto_region_name}')

role: arn:aws:iam::635837196364:role/service-role/AmazonSageMaker-ExecutionRole-20220427T210117
bucket: sagemaker-us-east-1-635837196364
session region: us-east-1


In [20]:
train_dataset = tokenized_train
test_dataset = datasets.concatenate_datasets([tokenized_val, tokenized_test])

Upload to S3 bucket

In [21]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()
s3_prefix = 'dataset/danmu_main'

#save data to S3
train_input_path = f's3://{session.default_bucket()}/{s3_prefix}/train'
#train_dataset.save_to_disk(train_input_path, fs = s3)
session.upload_data(path='data_csv/train.csv', bucket=session_bucket, key_prefix=s3_prefix+'/train')

test_input_path = f's3://{session.default_bucket()}/{s3_prefix}/test'
#test_dataset.save_to_disk(test_input_path, fs = s3)
session.upload_data(path='data_csv/test.csv', bucket=session_bucket, key_prefix=s3_prefix+'/test')

val_input_path = f's3://{session.default_bucket()}/{s3_prefix}/val'
session.upload_data(path='data_csv/val.csv', bucket=session_bucket, key_prefix=s3_prefix+'/val')


's3://sagemaker-us-east-1-635837196364/dataset/danmu_main/val/val.csv'

In [22]:
import sagemaker
from sagemaker.huggingface import HuggingFace

# gets role for executing training job

hyperparameters = {
    'model_name_or_path':'t5-small',
    'output_dir':'/opt/ml/model/t5_small',
    # add your remaining hyperparameters 
    # more info here https://github.com/huggingface/transformers/tree/v4.17.0/examples/pytorch/text-classification
    'max_seq_length':128,
    'per_device_train_batch_size' : 64,
    'learning_rate' : 2e-5,
    'num_train_epochs': 1,
    #train
    'do_train': True,
    #data
    'train_file': '/opt/ml/input/data/train/train.csv',
    #'test_file': '/opt/ml/input/data/train/test.csv',
    'validation_file': '/opt/ml/input/data/val/val.csv',
}


In [24]:
# git configuration to download our fine-tuning script
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.17.0'}

# creates Hugging Face estimator
huggingface_estimator = HuggingFace(
    entry_point='run_glue_t5.py',
    #entry_point='run_glue.py',
    #source_dir='./examples/pytorch/text-classification',
    source_dir = './scripts',
    instance_type='ml.g5.xlarge',
    instance_count=1,
    role=role,
    #it_config=git_config,
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    hyperparameters = hyperparameters
)


In [25]:

# starting the train job
huggingface_estimator.fit({'train': train_input_path, 'test': test_input_path, 'val':val_input_path})

2022-05-11 19:14:05 Starting - Starting the training job...
2022-05-11 19:14:32 Starting - Preparing the instances for trainingProfilerReport-1652296445: InProgress
.........
2022-05-11 19:15:52 Downloading - Downloading input data
2022-05-11 19:15:52 Training - Downloading the training image...........................
2022-05-11 19:20:32 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-11 19:20:31,297 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-11 19:20:31,319 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-11 19:20:31,326 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-11 19:20:31,938 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channe

UnexpectedStatusException: Error for Training job huggingface-pytorch-training-2022-05-11-19-14-04-508: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "ValueError: not enough values to unpack (expected 2, got 1)
 0%|          | 0/3796 [00:03<?, ?it/s]"
Command "/opt/conda/bin/python3.8 run_glue_t5.py --do_train True --learning_rate 2e-05 --max_seq_length 128 --model_name_or_path t5-small --num_train_epochs 1 --output_dir /opt/ml/model/t5_small --per_device_train_batch_size 64 --train_file /opt/ml/input/data/train/train.csv --validation_file /opt/ml/input/data/val/val.csv", exit code: 1

In [202]:
train_dataset[0]

{'text': '菲布理',
 'label': 2,
 'input_ids': [3, 2, 1],
 'attention_mask': [1, 1, 1]}